In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.tri import LinearTriInterpolator, CubicTriInterpolator, Triangulation

from pyquasar import load_mesh, FemDomain, BemDomain, FetiProblem, Coil2D

In [ ]:
def print_feti_information(problem):
  print(f'Feti problem: domains {len(problem.domains)}, dual size {problem.dual_size}, primal size {problem.primal_size}')
  print(f'Condition number estimation {problem.condition_number_estimate():.2f}')

def print_domains_information(domains):
  fem = [domain for domain in domains if isinstance(domain, FemDomain)]
  print(f'FEM domains {len(fem)}, elements {sum(d.element_count for d in fem)}, dof {sum(d.dof_count for d in fem)}')
  bem = [domain for domain in domains if isinstance(domain, BemDomain)]
  print(f'BEM domains {len(bem)}, elements {sum(d.element_count for d in bem)}, dof {sum(d.dof_count for d in bem)}')

coil_vertices = np.array([[-1, 0.5], [+1, 0.5], [+1, 2.5], [-1, 2.5]])
coil_vertices = [coil_vertices, coil_vertices - [0, 3], coil_vertices[::-1] + [7, 0], coil_vertices[::-1] + [7, -3]]
coil = Coil2D(coil_vertices)

mu0 = 4 * np.pi * 1e-7
mu = 1000 * mu0

#mesh = [data for data in load_mesh('/content/dipole2d_fine.msh', refineK=0)]
mesh = [data for data in load_mesh('dipole2d.geo', refineK=2)]
problem = FetiProblem([FemDomain(*data) for data in mesh])
print_domains_information(problem.domains)

mesh2 = [data for data in load_mesh('dipole2d.geo', refineK=3)]
problem2 = FetiProblem([FemDomain(*data) for data in mesh2])
print_domains_information(problem2.domains)

# Ploting

In [ ]:
def plot_scalar(problem, solutions, func, title, mesh=False):
  fig, ax = plt.subplots(figsize=(10, 8))
  ax.set_title(title)
  ax.set_xlim(-7, 9)
  ax.set_ylim(-8, 8)

  triangs = [Triangulation(domain.vertices[:, 0], domain.vertices[:, 1], domain.elements[0][1]) for domain in problem.domains]
  triangs += [Triangulation(t.x, -t.y, t.triangles) for t in triangs]

  interps = [LinearTriInterpolator(triang, sol) for triang, sol in zip(triangs, solutions * 2)]

  n = len(problem.domains)
  values = [func(triang.x, triang.y, interp, domain.material, sign) for triang, interp, domain, sign
            in zip(triangs, interps, problem.domains * 2, [1] * n + [-1] * n)]

  parts = []
  for triang, value in zip(triangs, values):
    parts.append(ax.tripcolor(triang, value, shading='gouraud', cmap='seismic'))

  vmin = min(part.get_array().min() for part in parts)
  vmax = max(part.get_array().max() for part in parts)
  norm = colors.Normalize(vmin=vmin, vmax=vmax)
  for p in parts:
      p.set_norm(norm)

  levels = np.linspace(vmin, vmax, 30)
  for triang, value in zip(triangs, values):
    ax.tricontour(triang, value, levels=levels)

  if mesh:
    for triang in triangs[:n]:
      ax.triplot(triang, linewidth=0.2)

  fig.colorbar(parts[0], ax=ax)
  plt.show()

def plot_vector(problem, solutions, func, title, mesh=False):
  fig, ax = plt.subplots(figsize=(10, 8))
  ax.set_title(title)
  ax.set_xlim(-7, 9)
  ax.set_ylim(-8, 8)

  triangs = [Triangulation(domain.vertices[:, 0], domain.vertices[:, 1], domain.elements[0][1]) for domain in problem.domains]
  triangs += [Triangulation(t.x, -t.y, t.triangles) for t in triangs]

  n = len(problem.domains)
  interps = [CubicTriInterpolator(triang, sol) for triang, sol in zip(triangs, solutions)]
  interps += [CubicTriInterpolator(triang, sol) for triang, sol in zip(triangs[n:], solutions)]

  X, Y = np.meshgrid(np.linspace(-7, 9, 40), np.linspace(-8, 8, 40))
  U = np.zeros_like(X)
  V = np.zeros_like(Y)

  values = []
  for triang, interp, domain, sign in zip(triangs, interps, problem.domains * 2, [1] * n + [-1] * n):
    values.append(np.linalg.norm(func(triang.x, triang.y, interp, domain.material, sign), axis=0))

    mask = triang.get_trifinder()(X, Y) != -1
    valX, valY = func(X[mask], Y[mask], interp, domain.material, sign)
    U[mask] = valX
    V[mask] = valY

  parts = []
  for triang, value in zip(triangs, values):
    parts.append(ax.tripcolor(triang, value, shading='gouraud', cmap='Purples'))

  vmin = min(part.get_array().min() for part in parts)
  vmax = max(part.get_array().max() for part in parts)
  norm = colors.Normalize(vmin=vmin, vmax=vmax)
  for p in parts:
      p.set_norm(norm)

  if mesh:
    for triang in triangs[:n]:
      ax.triplot(triang, linewidth=0.2)

  Bnorm = np.sqrt(U ** 2 + V ** 2)
  #bar = ax.quiver(X, Y, U/Bnorm, V/Bnorm, Bnorm, norm=norm, cmap='plasma')
  bar = ax.quiver(X, Y, U, V, Bnorm, norm=norm, cmap='plasma')

  fig.colorbar(bar, ax=ax)
  plt.show()

def plot_diff(problem1, solutions1, problem2, solutions2, func1, func2, title, mesh=False):
  fig, ax = plt.subplots(figsize=(10, 8))
  ax.set_title(title)
  ax.set_xlim(-7, 9)
  ax.set_ylim(-8, 8)

  triangs = [Triangulation(domain.vertices[:, 0], domain.vertices[:, 1], domain.elements[0][1]) for domain in problem1.domains]
  triangs += [Triangulation(t.x, -t.y, t.triangles) for t in triangs]

  triangs2 = [Triangulation(domain.vertices[:, 0], domain.vertices[:, 1], domain.elements[0][1]) for domain in problem2.domains]
  triangs2 += [Triangulation(t.x, -t.y, t.triangles) for t in triangs2]

  interps1 = [LinearTriInterpolator(triang, sol) for triang, sol in zip(triangs, solutions1 * 2)]
  interps2 = [LinearTriInterpolator(triang, sol) for triang, sol in zip(triangs2, solutions2 * 2)]

  n = len(problem.domains)
  values1 = [func1(triang.x, triang.y, interp, domain.material, sign) for triang, interp, domain, sign
            in zip(triangs, interps1, problem.domains * 2, [1] * n + [-1] * n)]
  values2 = [func2(triang.x, triang.y, interp, domain.material, sign) for triang, interp, domain, sign
            in zip(triangs, interps2, problem.domains * 2, [1] * n + [-1] * n)]
  max_val_norm = max(np.max(np.sqrt(x ** 2 + y ** 2)) for x, y in values2)

  diffs = [np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)/max_val_norm for (x1, y1), (x2, y2) in zip(values1, values2)]

  parts = []
  for triang, diff in zip(triangs, diffs):
    parts.append(ax.tripcolor(triang, diff, shading='gouraud', cmap='turbo'))

  #vmin = min(part.get_array().min() for part in parts)
  #vmax = max(part.get_array().max() for part in parts)
  #norm = colors.LogNorm(vmin=vmin, vmax=vmax)
  norm = colors.LogNorm(vmin=1e-6, vmax=1)
  for p in parts:
      p.set_norm(norm)

  if mesh:
    for triang in triangs[:n]:
      ax.triplot(triang, linewidth=0.2)

  fig.colorbar(parts[0], ax=ax)
  plt.show()

# Total $A$ potential

$\sum\limits_i \left(\frac{1}{\mu} \nabla A, \nabla v\right)_{\Omega_i} = \sum\limits_i  \left(J, v\right)_{\Omega_i}$

In [ ]:
materials = { 'coil_pos': { 'coeff': 1.0/mu0, 'coil_pos': 1 },
              'coil_neg': { 'coeff': 1.0/mu0, 'coil_neg': -1 },
              'air': { 'coeff': 1.0/mu0 },
              'steel': { 'coeff': 1.0/mu }
            }

problem.assembly('dirichlet', materials)
print_feti_information(problem)
problem.decompose()
solutions = problem.solve()

In [ ]:
problem2.assembly('dirichlet', materials)
print_feti_information(problem2)
problem2.decompose()
best_sol = problem2.solve()

In [ ]:
def calcB_from_totalA(x, y, interp, mat, sign):
  dx, dy = interp.gradient(x, y)
  return dy, -dx

calcB = calcB_from_totalA

#plot_scalar(problem, solutions, lambda x, y, interp, mat, sign: interp(x, y), "$A_{total}$")
plot_vector(problem, solutions, calcB_from_totalA, "$B(A_{total})$", mesh=True)
plot_diff(problem, solutions, problem2, best_sol, calcB_from_totalA, calcB, "$||B(A_{total}) - B_{dbl}(A_{total})||/||B_{dbl}(A_{total})||_\\infty$")

# Reducted $A$ potential
$\sum\limits_i \left(\frac{1}{\mu} \nabla A, \nabla v\right)_{\Omega_i} = - \sum\limits_i \left(\frac{1}{\mu} \frac{\partial A_e}{\partial n}, v\right)_{\partial\Omega_i}$

$-\Delta A_e = \mu_0 J$

In [ ]:
materials = { 'steel': { 'coeff': 1.0/mu, 'gap': lambda p, n: -np.sum(mu0 * coil.calc_gradA(p) * n, axis=-1)/mu },
              'air': { 'coeff': 1.0/mu0, 'gap': lambda p, n: -np.sum(mu0 * coil.calc_gradA(p) * n, axis=-1)/mu0 },
              'coil_pos': { 'coeff': 1.0/mu0 },
              'coil_neg': { 'coeff': 1.0/mu0 }
            }

problem.assembly('dirichlet', materials)
print_feti_information(problem)
problem.decompose()
solutions = problem.solve()

In [ ]:
def calcB_from_reductedA(x, y, interp, mat, sign):
  dx, dy = interp.gradient(x, y)
  t = mu0 * coil.calc_gradA(np.stack((x, y), axis=-1))
  return dy + t[..., 1], -(dx + t[..., 0])

#plot_scalar(problem, solutions, lambda x, y, interp, mat, sign: interp(x, y), "$A_{reducted}$")
#plot_vector(problem, solutions, calcB_from_reductedA, "$B(A_{reducted})$")
plot_diff(problem, solutions, problem2, best_sol, calcB_from_reductedA, calcB, "$||B(A_{reducted}) - B_{dbl}(A_{total})||/||B_{dbl}(A_{total})||_\\infty$")

# Total and reducted $A$ potentials

$\sum\limits_i \left(\frac{1}{\mu} \nabla A, \nabla v\right)_{\Omega_i} = - \sum\limits_{i\in\Omega_r} \left(\frac{1}{\mu} \frac{\partial A_e}{\partial n}, v\right)_{\partial\Omega_i}$

$\left[A\right]_{\partial \Omega_r \cap \partial \Omega_t} = A_e, -\Delta A_e = \mu_0 J$

In [ ]:
materials = { 'steel': { 'coeff': 1.0/mu },
              'air': { 'coeff': 1.0/mu0, 'gap': lambda p, n: -np.sum(mu0 * coil.calc_gradA(p) * n, axis=-1)/mu0 },
              'coil_pos': { 'coeff': 1.0/mu0 },
              'coil_neg': { 'coeff': 1.0/mu0 }
            }

problem.assembly('dirichlet', materials)
print_feti_information(problem)
problem.add_skeleton_projection(lambda p, n: -mu0 * coil.calcA(p), {'air': {'gap'}})
problem.decompose()
solutions = problem.solve()

In [ ]:
def calcB_from_mixedA(x, y, interp, mat, sign):
  dx, dy = interp.gradient(x, y)
  if mat == 'steel':
    return dy, -dx
  t = mu0 * coil.calc_gradA(np.stack((x, y), axis=-1))
  return dy +  t[..., 1], -(dx + t[..., 0])

#plot_scalar(problem, solutions, lambda x, y, interp, mat, sign: interp(x, y), "$A_{mixed}$")
#plot_vector(problem, solutions, calcB_from_mixedA, "$B(A_{mixed})$")
plot_diff(problem, solutions, problem2, best_sol, calcB_from_mixedA, calcB, "$||B(A_{mixed}) - B_{dbl}(A_{total})||/||B_{dbl}(A_{total})||_\\infty$")

# Reducted $\varphi$ potential

$\sum\limits_i \left(\mu \nabla \varphi, \nabla v\right)_{\Omega_i} = \sum\limits_i \left(\mu H_e \cdot n, v\right)_{\partial\Omega_i}$

$\nabla \times H_e = J, \nabla \cdot H_e = 0$

$H_e = \nabla \times \left(A_e e_z \right), -\Delta A_e = J$

In [ ]:
materials = { 'steel': { 'coeff': mu, 'gap': lambda p, n: mu * np.sum(coil.calc_rotA(p) * n, axis=-1) },
              'air': { 'coeff': mu0, 'gap': lambda p, n: mu0 * np.sum(coil.calc_rotA(p) * n, axis=-1) },
              'coil_pos': { 'coeff': mu0 },
              'coil_neg': { 'coeff': mu0 }
            }

problem.assembly('neumann', materials)
print_feti_information(problem)
problem.decompose()
solutions = problem.solve()

In [ ]:
def calcB_from_reductedPhi(x, y, interp, mat, sign):
  dx, dy = interp.gradient(x, y)
  t = coil.calc_rotA(np.stack((x, y), axis=-1))
  if mat == 'steel':
    return mu * (t[..., 0] - sign * dx), mu * (t[..., 1] - sign * dy)
  return mu0 * (t[..., 0] - sign * dx), mu0 * (t[..., 1] - sign * dy)

#plot_scalar(problem, solutions, lambda x, y, interp, mat, sign: sign * interp(x, y), "$\\varphi_{reducted}$", mesh=True)
#plot_vector(problem, solutions, calcB_from_reductedPhi, "$B(\\varphi_{reducted})$", mesh=True)
plot_diff(problem, solutions, problem2, best_sol, calcB_from_reductedPhi, calcB, "$||B(\\varphi_{reducted}) - B_{dbl}(A_{total})||/||B_{dbl}(A_{total})||_\\infty$")

# Total and reducted $\varphi$ potentials
$\sum\limits_i \left(\mu \nabla \varphi, \nabla v\right)_{\Omega_i} = \sum\limits_{i \in \Omega_r} \left(\mu H_e \cdot n, v\right)_{\partial\Omega_i}$

$\left[\nabla \varphi \times n \right]_{\partial \Omega_r \cap \partial \Omega_t} = H_e \times n, \nabla \times H_e = J, \nabla \cdot H_e = 0$

$H_e = \nabla \times \left(A_e e_z \right), -\Delta A_e = J$

In [ ]:
materials = { 'steel': { 'coeff': mu },
              'air': { 'coeff': mu0, 'gap': lambda p, n: mu0 * np.sum(coil.calc_rotA(p) * n, axis=-1) },
              'coil_pos': { 'coeff': mu0 },
              'coil_neg': { 'coeff': mu0 }
            }

problem.assembly('neumann', materials)
print_feti_information(problem)
proj = problem.add_skeleton_projection(lambda p, n: coil.calc_rotA(p), {'air': {'gap'}}, grad = True)
problem.decompose()
solutions = problem.solve()

In [ ]:
def calcB_from_mixedPhi(x, y, interp, mat, sign):
  dx, dy = interp.gradient(x, y)
  if mat == 'steel':
    return -mu * sign * dx, -mu * sign * dy
  t = coil.calc_rotA(np.stack((x, y), axis=-1))
  return mu0 * (t[..., 0] - sign * dx), mu0 * (t[..., 1] - sign * dy)

#plot_scalar(problem, solutions, lambda x, y, interp, mat, sign: sign * interp(x, y), "$\\varphi_{mixed}$")
#plot_vector(problem, solutions, calcB_from_mixedPhi, "$B(\\varphi_{mixed})$")
plot_diff(problem, solutions, problem2, best_sol, calcB_from_mixedPhi, calcB, "$||B(\\varphi_{mixed}) - B_{dbl}(A_{total})||/||B_{dbl}(A_{total})||_\\infty$")